# 🎯 Prediksi Employee Attrition dengan Random Forest

## Deskripsi Project
Notebook ini bertujuan untuk membangun model Machine Learning yang dapat memprediksi kemungkinan seorang karyawan akan resign (attrition) berdasarkan berbagai faktor seperti gaji, jarak rumah, kepuasan kerja, dan lain-lain.

## Metodologi
Kita akan melakukan **eksperimen bertahap** untuk menemukan model yang optimal:
1. **Model 1 (Full Features)** - Menggunakan semua 31 fitur
2. **Model 2 (Reduced Features)** - Mengurangi fitur berdasarkan analisis korelasi dan importance
3. **Model 3 (Minimal Features)** - Hanya menggunakan fitur dengan importance tertinggi

## Dataset
- **Sumber**: IBM HR Analytics Employee Attrition Dataset
- **Total Sampel**: 1,470 karyawan
- **Target**: Attrition (Yes/No)

---
**Author**: Data Mining Project  
**Date**: November 2025

In [ ]:
---
# 📚 BAGIAN 1: IMPORT LIBRARY

Pada bagian ini kita mengimport semua library yang diperlukan untuk:
- **Pandas & NumPy**: Manipulasi dan analisis data
- **Matplotlib & Seaborn**: Visualisasi data
- **Scikit-learn**: Machine learning (preprocessing, modeling, evaluasi)

In [ ]:
# Import Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import time

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Settings
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✅ Semua library berhasil diimport!")

---
# 📊 BAGIAN 2: LOAD DATA

Kita akan memuat dataset HR Employee Attrition dari IBM. Dataset ini berisi informasi 1,470 karyawan dengan 35 kolom fitur.

In [ ]:
# Load Dataset
# Untuk Google Colab, upload file terlebih dahulu atau gunakan URL
df = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')

print("="*60)
print("INFORMASI DATASET")
print("="*60)
print(f"\n📁 Jumlah Baris   : {df.shape[0]}")
print(f"📁 Jumlah Kolom   : {df.shape[1]}")
print(f"\n📋 Kolom-kolom dalam dataset:")
print(df.columns.tolist())

In [ ]:
# Lihat 5 baris pertama data
print("📊 Preview Data (5 Baris Pertama):")
df.head()

In [ ]:
# Informasi tipe data dan missing values
print("📊 Informasi Tipe Data:")
print(df.dtypes.value_counts())
print(f"\n❓ Missing Values: {df.isnull().sum().sum()}")

In [ ]:
# Distribusi Target Variable (Attrition)
print("📊 Distribusi Target Variable (Attrition):")
print(df['Attrition'].value_counts())
print(f"\n📈 Persentase Attrition:")
print(f"   - No (Tidak Resign) : {(df['Attrition']=='No').sum()/len(df)*100:.1f}%")
print(f"   - Yes (Resign)      : {(df['Attrition']=='Yes').sum()/len(df)*100:.1f}%")

# Visualisasi
fig, ax = plt.subplots(figsize=(8, 5))
colors = ['#4ECDC4', '#FF6B6B']
df['Attrition'].value_counts().plot(kind='bar', color=colors, edgecolor='black', linewidth=2)
plt.title('Distribusi Employee Attrition', fontsize=14, fontweight='bold')
plt.xlabel('Attrition', fontsize=12)
plt.ylabel('Jumlah Karyawan', fontsize=12)
plt.xticks(rotation=0)
for i, v in enumerate(df['Attrition'].value_counts()):
    plt.text(i, v + 20, str(v), ha='center', fontweight='bold', fontsize=12)
plt.tight_layout()
plt.show()

---
# 🔧 BAGIAN 3: PREPROCESSING DATA

Pada tahap preprocessing, kita akan:
1. Menghapus kolom yang tidak berguna (konstanta/tidak relevan)
2. Memisahkan fitur (X) dan target (y)
3. Membagi data menjadi training dan testing set

In [ ]:
# Cek kolom dengan nilai konstan (tidak berguna untuk prediksi)
print("🔍 Mencari kolom dengan nilai konstan...")
for col in df.columns:
    if df[col].nunique() == 1:
        print(f"   ❌ Kolom '{col}' hanya punya 1 nilai unik: {df[col].unique()[0]}")

print("\n🔍 Kolom yang akan dihapus:")
print("   - Over18: Semua bernilai 'Y' (konstanta)")
print("   - EmployeeCount: Semua bernilai 1 (konstanta)")  
print("   - StandardHours: Semua bernilai 80 (konstanta)")
print("   - EmployeeNumber: Hanya ID, tidak relevan untuk prediksi")

In [ ]:
# Hapus kolom yang tidak berguna
df_clean = df.drop(['Over18', 'EmployeeCount', 'StandardHours', 'EmployeeNumber'], axis=1)

print("="*60)
print("HASIL PREPROCESSING")
print("="*60)
print(f"\n📊 Sebelum: {df.shape[1]} kolom")
print(f"📊 Setelah: {df_clean.shape[1]} kolom")
print(f"📊 Kolom dihapus: {df.shape[1] - df_clean.shape[1]} kolom")

In [ ]:
# Pisahkan Fitur (X) dan Target (y)
X = df_clean.drop('Attrition', axis=1)
y = df_clean['Attrition'].map({'Yes': 1, 'No': 0})

print("="*60)
print("PEMISAHAN FITUR DAN TARGET")
print("="*60)
print(f"\n📊 Jumlah Fitur (X): {X.shape[1]} kolom")
print(f"📊 Jumlah Sampel   : {X.shape[0]} baris")
print(f"\n📊 Distribusi Target (y):")
print(f"   - 0 (Tidak Resign): {(y==0).sum()} ({(y==0).sum()/len(y)*100:.1f}%)")
print(f"   - 1 (Resign)      : {(y==1).sum()} ({(y==1).sum()/len(y)*100:.1f}%)")

In [ ]:
# Identifikasi tipe fitur
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print("="*60)
print("TIPE FITUR")
print("="*60)
print(f"\n📊 Fitur Kategorikal ({len(categorical_features)} kolom):")
print(f"   {categorical_features}")
print(f"\n📊 Fitur Numerik ({len(numeric_features)} kolom):")
print(f"   {numeric_features}")

In [ ]:
# Split Data: 80% Training, 20% Testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y  # Menjaga proporsi kelas
)

print("="*60)
print("PEMBAGIAN DATA TRAINING & TESTING")
print("="*60)
print(f"\n📊 Data Training: {X_train.shape[0]} sampel ({X_train.shape[0]/len(X)*100:.0f}%)")
print(f"📊 Data Testing : {X_test.shape[0]} sampel ({X_test.shape[0]/len(X)*100:.0f}%)")
print(f"\n✅ Data siap untuk modeling!")

---
---
# 🤖 MODEL 1: FULL FEATURES (31 Fitur)

## Deskripsi
Pada percobaan pertama, kita akan menggunakan **SEMUA 31 fitur** yang tersedia setelah preprocessing. Tujuannya adalah untuk mendapatkan baseline performance dan melihat fitur mana yang paling berpengaruh.

## Fitur yang Digunakan
Semua fitur dari dataset (kecuali yang sudah dihapus)

In [ ]:
# MODEL 1: Menggunakan semua 31 fitur
print("="*70)
print("MODEL 1: FULL FEATURES (31 Fitur)")
print("="*70)

# Daftar semua fitur untuk Model 1
features_model1 = X.columns.tolist()

print(f"\n📊 Jumlah Fitur: {len(features_model1)}")
print(f"\n📋 Daftar Fitur:")
for i, feat in enumerate(features_model1, 1):
    print(f"   {i:2d}. {feat}")

In [ ]:
# Persiapan data untuk Model 1
X_train_m1 = X_train[features_model1].copy()
X_test_m1 = X_test[features_model1].copy()

# Identifikasi fitur kategorikal dan numerik
cat_features_m1 = X_train_m1.select_dtypes(include=['object']).columns.tolist()
num_features_m1 = X_train_m1.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"📊 Fitur Kategorikal: {len(cat_features_m1)}")
print(f"📊 Fitur Numerik    : {len(num_features_m1)}")

In [ ]:
# Membuat Pipeline untuk Model 1
# Pipeline menggabungkan preprocessing (One-Hot Encoding) dan model (Random Forest)

# Preprocessing: One-Hot Encoding untuk fitur kategorikal
preprocessor_m1 = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_features_m1)
    ],
    remainder='passthrough'  # Fitur numerik tetap dipertahankan
)

# Pipeline lengkap: Preprocessing + Random Forest
pipeline_model1 = Pipeline([
    ('preprocessor', preprocessor_m1),
    ('classifier', RandomForestClassifier(
        n_estimators=300,      # Jumlah pohon
        max_depth=15,          # Kedalaman maksimum pohon
        min_samples_split=10,  # Minimum sampel untuk split
        min_samples_leaf=5,    # Minimum sampel di leaf
        random_state=42,
        n_jobs=-1              # Gunakan semua CPU
    ))
])

print("✅ Pipeline Model 1 berhasil dibuat!")
print(f"\n🔧 Konfigurasi Random Forest:")
print(f"   - n_estimators    : 300")
print(f"   - max_depth       : 15")
print(f"   - min_samples_split: 10")
print(f"   - min_samples_leaf : 5")

In [ ]:
# Training Model 1
print("="*70)
print("TRAINING MODEL 1")
print("="*70)

start_time = time.time()
pipeline_model1.fit(X_train_m1, y_train)
training_time_m1 = time.time() - start_time

print(f"\n✅ Training selesai dalam {training_time_m1:.4f} detik")

In [ ]:
# Prediksi Model 1
y_pred_train_m1 = pipeline_model1.predict(X_train_m1)
y_pred_test_m1 = pipeline_model1.predict(X_test_m1)

# Hitung Akurasi
train_acc_m1 = accuracy_score(y_train, y_pred_train_m1)
test_acc_m1 = accuracy_score(y_test, y_pred_test_m1)

print("="*70)
print("EVALUASI MODEL 1: FULL FEATURES")
print("="*70)
print(f"\n📊 Akurasi Training : {train_acc_m1*100:.2f}%")
print(f"📊 Akurasi Testing  : {test_acc_m1*100:.2f}%")
print(f"📊 Overfitting      : {(train_acc_m1 - test_acc_m1)*100:.2f}%")

In [ ]:
# Confusion Matrix Model 1
cm_m1 = confusion_matrix(y_test, y_pred_test_m1)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm_m1, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Tidak Resign', 'Resign'],
            yticklabels=['Tidak Resign', 'Resign'],
            annot_kws={'size': 16, 'weight': 'bold'},
            linewidths=2, linecolor='black')

plt.title(f'Confusion Matrix - Model 1 (Full Features)\nAkurasi: {test_acc_m1*100:.2f}%', 
          fontsize=14, fontweight='bold', pad=15)
plt.xlabel('Prediksi', fontsize=12, fontweight='bold')
plt.ylabel('Aktual', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"\n📊 Detail Confusion Matrix:")
print(f"   - True Negative  (TN): {cm_m1[0][0]} - Prediksi Tidak Resign, Aktual Tidak Resign")
print(f"   - False Positive (FP): {cm_m1[0][1]} - Prediksi Resign, Aktual Tidak Resign")
print(f"   - False Negative (FN): {cm_m1[1][0]} - Prediksi Tidak Resign, Aktual Resign")
print(f"   - True Positive  (TP): {cm_m1[1][1]} - Prediksi Resign, Aktual Resign")

In [ ]:
# Classification Report Model 1
print("="*70)
print("CLASSIFICATION REPORT - MODEL 1")
print("="*70)
print(classification_report(y_test, y_pred_test_m1, 
                           target_names=['Tidak Resign', 'Resign']))

In [ ]:
# Feature Importance Model 1
# Ambil nama fitur setelah One-Hot Encoding
encoder_m1 = pipeline_model1.named_steps['preprocessor'].named_transformers_['cat']
cat_feature_names_m1 = encoder_m1.get_feature_names_out(cat_features_m1).tolist()
all_feature_names_m1 = cat_feature_names_m1 + num_features_m1

# Ambil importance dari Random Forest
importances_m1 = pipeline_model1.named_steps['classifier'].feature_importances_

# Buat DataFrame dan sort
feat_imp_m1 = pd.DataFrame({
    'Feature': all_feature_names_m1,
    'Importance': importances_m1
}).sort_values('Importance', ascending=False)

# Tampilkan Top 15
print("="*70)
print("TOP 15 FEATURE IMPORTANCE - MODEL 1")
print("="*70)
print(feat_imp_m1.head(15).to_string(index=False))

In [ ]:
# Visualisasi Feature Importance Model 1 (Top 15)
fig, ax = plt.subplots(figsize=(12, 8))

top15_m1 = feat_imp_m1.head(15).sort_values('Importance', ascending=True)
colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(top15_m1)))

bars = ax.barh(top15_m1['Feature'], top15_m1['Importance']*100, 
               color=colors, edgecolor='black', linewidth=1.5)

for bar, val in zip(bars, top15_m1['Importance']*100):
    ax.text(val + 0.3, bar.get_y() + bar.get_height()/2, 
            f'{val:.2f}%', va='center', fontweight='bold', fontsize=10)

ax.set_xlabel('Importance (%)', fontsize=12, fontweight='bold')
ax.set_title('Feature Importance - Model 1 (Full Features)\nTop 15 Fitur Paling Berpengaruh', 
             fontsize=14, fontweight='bold', pad=15)
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

### 📝 Kesimpulan Model 1 (Full Features)

**Hasil:**
- Akurasi Testing: ~84%
- Menggunakan 31 fitur
- Terdapat overfitting (akurasi training > testing)

**Insight dari Feature Importance:**
Dari hasil di atas, kita melihat beberapa fitur yang paling berpengaruh seperti:
- **OverTime** (lembur)
- **MonthlyIncome** (gaji bulanan)
- **Age** (usia)
- **TotalWorkingYears** (total pengalaman kerja)

**Pertanyaan:**
> Apakah kita bisa mendapatkan akurasi yang sama dengan fitur yang lebih sedikit?
> Ini akan kita coba di Model 2 dengan mengurangi fitur yang kurang penting.

---
---
# 🤖 MODEL 2: REDUCED FEATURES (11 Fitur)

## Deskripsi
Berdasarkan hasil Feature Importance dari Model 1, kita akan **mengurangi jumlah fitur** menjadi 11 fitur yang paling berpengaruh. Tujuannya adalah:
1. Mengurangi kompleksitas model
2. Menghindari overfitting
3. Mempercepat waktu training

## Kriteria Pemilihan Fitur
Fitur dipilih berdasarkan:
- Feature Importance > 3%
- Fitur yang secara bisnis masuk akal mempengaruhi keputusan resign

In [ ]:
# MODEL 2: Reduced Features (11 Fitur)
print("="*70)
print("MODEL 2: REDUCED FEATURES (11 Fitur)")
print("="*70)

# Daftar 11 fitur yang dipilih berdasarkan importance dari Model 1
features_model2 = [
    'OverTime',              # Lembur - pengaruh tertinggi
    'MonthlyIncome',         # Gaji bulanan
    'Age',                   # Usia karyawan
    'YearsAtCompany',        # Lama bekerja di perusahaan
    'JobLevel',              # Level jabatan
    'TotalWorkingYears',     # Total pengalaman kerja
    'StockOptionLevel',      # Level opsi saham
    'DistanceFromHome',      # Jarak dari rumah
    'EnvironmentSatisfaction', # Kepuasan lingkungan kerja
    'YearsInCurrentRole',    # Lama di posisi sekarang
    'YearsWithCurrManager'   # Lama dengan manajer sekarang
]

print(f"\n📊 Jumlah Fitur: {len(features_model2)} (dari 31 → 11)")
print(f"📊 Pengurangan : {(1 - len(features_model2)/31)*100:.1f}%")
print(f"\n📋 Daftar Fitur yang Dipilih:")
for i, feat in enumerate(features_model2, 1):
    print(f"   {i:2d}. {feat}")

In [ ]:
# Persiapan data untuk Model 2
X_train_m2 = X_train[features_model2].copy()
X_test_m2 = X_test[features_model2].copy()

# Identifikasi fitur kategorikal dan numerik untuk Model 2
cat_features_m2 = X_train_m2.select_dtypes(include=['object']).columns.tolist()
num_features_m2 = X_train_m2.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"📊 Fitur Kategorikal: {len(cat_features_m2)} → {cat_features_m2}")
print(f"📊 Fitur Numerik    : {len(num_features_m2)}")

In [ ]:
# Membuat Pipeline untuk Model 2
preprocessor_m2 = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_features_m2)
    ],
    remainder='passthrough'
)

pipeline_model2 = Pipeline([
    ('preprocessor', preprocessor_m2),
    ('classifier', RandomForestClassifier(
        n_estimators=300,
        max_depth=15,
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=42,
        n_jobs=-1
    ))
])

print("✅ Pipeline Model 2 berhasil dibuat!")

In [ ]:
# Training Model 2
print("="*70)
print("TRAINING MODEL 2")
print("="*70)

start_time = time.time()
pipeline_model2.fit(X_train_m2, y_train)
training_time_m2 = time.time() - start_time

print(f"\n✅ Training selesai dalam {training_time_m2:.4f} detik")
print(f"📊 Perbandingan waktu dengan Model 1: {(training_time_m2/training_time_m1)*100:.1f}%")

In [ ]:
# Prediksi Model 2
y_pred_train_m2 = pipeline_model2.predict(X_train_m2)
y_pred_test_m2 = pipeline_model2.predict(X_test_m2)

# Hitung Akurasi
train_acc_m2 = accuracy_score(y_train, y_pred_train_m2)
test_acc_m2 = accuracy_score(y_test, y_pred_test_m2)

print("="*70)
print("EVALUASI MODEL 2: REDUCED FEATURES")
print("="*70)
print(f"\n📊 Akurasi Training : {train_acc_m2*100:.2f}%")
print(f"📊 Akurasi Testing  : {test_acc_m2*100:.2f}%")
print(f"📊 Overfitting      : {(train_acc_m2 - test_acc_m2)*100:.2f}%")
print(f"\n📊 Perbandingan dengan Model 1:")
print(f"   - Model 1: {test_acc_m1*100:.2f}% (31 fitur)")
print(f"   - Model 2: {test_acc_m2*100:.2f}% (11 fitur)")
print(f"   - Selisih: {(test_acc_m2 - test_acc_m1)*100:.2f}%")

In [ ]:
# Confusion Matrix Model 2
cm_m2 = confusion_matrix(y_test, y_pred_test_m2)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm_m2, annot=True, fmt='d', cmap='Greens', 
            xticklabels=['Tidak Resign', 'Resign'],
            yticklabels=['Tidak Resign', 'Resign'],
            annot_kws={'size': 16, 'weight': 'bold'},
            linewidths=2, linecolor='black')

plt.title(f'Confusion Matrix - Model 2 (Reduced Features)\nAkurasi: {test_acc_m2*100:.2f}%', 
          fontsize=14, fontweight='bold', pad=15)
plt.xlabel('Prediksi', fontsize=12, fontweight='bold')
plt.ylabel('Aktual', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"\n📊 Detail Confusion Matrix:")
print(f"   - True Negative  (TN): {cm_m2[0][0]}")
print(f"   - False Positive (FP): {cm_m2[0][1]}")
print(f"   - False Negative (FN): {cm_m2[1][0]}")
print(f"   - True Positive  (TP): {cm_m2[1][1]}")

In [ ]:
# Classification Report Model 2
print("="*70)
print("CLASSIFICATION REPORT - MODEL 2")
print("="*70)
print(classification_report(y_test, y_pred_test_m2, 
                           target_names=['Tidak Resign', 'Resign']))

In [ ]:
# Feature Importance Model 2
encoder_m2 = pipeline_model2.named_steps['preprocessor'].named_transformers_['cat']
cat_feature_names_m2 = encoder_m2.get_feature_names_out(cat_features_m2).tolist()
all_feature_names_m2 = cat_feature_names_m2 + num_features_m2

importances_m2 = pipeline_model2.named_steps['classifier'].feature_importances_

feat_imp_m2 = pd.DataFrame({
    'Feature': all_feature_names_m2,
    'Importance': importances_m2
}).sort_values('Importance', ascending=False)

print("="*70)
print("FEATURE IMPORTANCE - MODEL 2")
print("="*70)
print(feat_imp_m2.to_string(index=False))

In [ ]:
# Visualisasi Feature Importance Model 2
fig, ax = plt.subplots(figsize=(10, 6))

top_m2 = feat_imp_m2.sort_values('Importance', ascending=True)
colors = plt.cm.Greens(np.linspace(0.4, 0.9, len(top_m2)))

bars = ax.barh(top_m2['Feature'], top_m2['Importance']*100, 
               color=colors, edgecolor='black', linewidth=1.5)

for bar, val in zip(bars, top_m2['Importance']*100):
    ax.text(val + 0.3, bar.get_y() + bar.get_height()/2, 
            f'{val:.2f}%', va='center', fontweight='bold', fontsize=10)

ax.set_xlabel('Importance (%)', fontsize=12, fontweight='bold')
ax.set_title('Feature Importance - Model 2 (Reduced Features)\n11 Fitur Terpilih', 
             fontsize=14, fontweight='bold', pad=15)
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

### 📝 Kesimpulan Model 2 (Reduced Features)

**Hasil:**
- Akurasi Testing: ~84% (hampir sama dengan Model 1)
- Menggunakan 11 fitur (pengurangan 65% dari Model 1)
- Waktu training lebih cepat

**Insight:**
Model 2 berhasil mempertahankan akurasi yang hampir sama dengan Model 1, padahal hanya menggunakan 35% fitur dari Model 1. Ini menunjukkan bahwa banyak fitur di Model 1 yang redundan/tidak terlalu berpengaruh.

**Pertanyaan:**
> Bisakah kita mengurangi fitur lebih jauh lagi?
> Di Model 3, kita akan mencoba hanya dengan 7 fitur yang paling penting.

---
---
# 🤖 MODEL 3: MINIMAL FEATURES (7 Fitur)

## Deskripsi
Ini adalah eksperimen terakhir, di mana kita hanya menggunakan **7 fitur dengan importance tertinggi**. Tujuannya adalah menciptakan model yang:
1. **Sederhana** - mudah diinterpretasi
2. **Efisien** - cepat dalam prediksi
3. **Praktis** - hanya membutuhkan sedikit data input

## Kriteria Pemilihan Fitur
Fitur dengan importance ≥ 7% dari Model 2

In [ ]:
# MODEL 3: Minimal Features (7 Fitur)
print("="*70)
print("MODEL 3: MINIMAL FEATURES (7 Fitur)")
print("="*70)

# Daftar 7 fitur paling penting berdasarkan hasil Model 2
features_model3 = [
    'OverTime',              # Lembur - faktor terbesar resign
    'MonthlyIncome',         # Gaji bulanan
    'Age',                   # Usia karyawan
    'TotalWorkingYears',     # Total pengalaman kerja
    'DistanceFromHome',      # Jarak dari rumah
    'StockOptionLevel',      # Level opsi saham (benefit)
    'EnvironmentSatisfaction' # Kepuasan lingkungan kerja
]

print(f"\n📊 Jumlah Fitur: {len(features_model3)} (dari 31 → 7)")
print(f"📊 Pengurangan : {(1 - len(features_model3)/31)*100:.1f}%")
print(f"\n📋 Daftar Fitur Minimal:")
for i, feat in enumerate(features_model3, 1):
    print(f"   {i}. {feat}")

In [ ]:
# Persiapan data untuk Model 3
X_train_m3 = X_train[features_model3].copy()
X_test_m3 = X_test[features_model3].copy()

# Identifikasi fitur kategorikal dan numerik untuk Model 3
cat_features_m3 = X_train_m3.select_dtypes(include=['object']).columns.tolist()
num_features_m3 = X_train_m3.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"📊 Fitur Kategorikal: {len(cat_features_m3)} → {cat_features_m3}")
print(f"📊 Fitur Numerik    : {len(num_features_m3)} → {num_features_m3}")

In [ ]:
# Membuat Pipeline untuk Model 3
preprocessor_m3 = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_features_m3)
    ],
    remainder='passthrough'
)

pipeline_model3 = Pipeline([
    ('preprocessor', preprocessor_m3),
    ('classifier', RandomForestClassifier(
        n_estimators=300,
        max_depth=15,
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=42,
        n_jobs=-1
    ))
])

print("✅ Pipeline Model 3 berhasil dibuat!")

In [ ]:
# Training Model 3
print("="*70)
print("TRAINING MODEL 3")
print("="*70)

start_time = time.time()
pipeline_model3.fit(X_train_m3, y_train)
training_time_m3 = time.time() - start_time

print(f"\n✅ Training selesai dalam {training_time_m3:.4f} detik")
print(f"📊 Perbandingan waktu:")
print(f"   - Model 1 (31 fitur): {training_time_m1:.4f} detik")
print(f"   - Model 2 (11 fitur): {training_time_m2:.4f} detik")
print(f"   - Model 3 (7 fitur) : {training_time_m3:.4f} detik")

In [ ]:
# Prediksi Model 3
y_pred_train_m3 = pipeline_model3.predict(X_train_m3)
y_pred_test_m3 = pipeline_model3.predict(X_test_m3)

# Hitung Akurasi
train_acc_m3 = accuracy_score(y_train, y_pred_train_m3)
test_acc_m3 = accuracy_score(y_test, y_pred_test_m3)

print("="*70)
print("EVALUASI MODEL 3: MINIMAL FEATURES")
print("="*70)
print(f"\n📊 Akurasi Training : {train_acc_m3*100:.2f}%")
print(f"📊 Akurasi Testing  : {test_acc_m3*100:.2f}%")
print(f"📊 Overfitting      : {(train_acc_m3 - test_acc_m3)*100:.2f}%")
print(f"\n📊 Perbandingan Semua Model:")
print(f"   - Model 1 (31 fitur): {test_acc_m1*100:.2f}%")
print(f"   - Model 2 (11 fitur): {test_acc_m2*100:.2f}%")
print(f"   - Model 3 (7 fitur) : {test_acc_m3*100:.2f}%")

In [ ]:
# Confusion Matrix Model 3
cm_m3 = confusion_matrix(y_test, y_pred_test_m3)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm_m3, annot=True, fmt='d', cmap='Oranges', 
            xticklabels=['Tidak Resign', 'Resign'],
            yticklabels=['Tidak Resign', 'Resign'],
            annot_kws={'size': 16, 'weight': 'bold'},
            linewidths=2, linecolor='black')

plt.title(f'Confusion Matrix - Model 3 (Minimal Features)\nAkurasi: {test_acc_m3*100:.2f}%', 
          fontsize=14, fontweight='bold', pad=15)
plt.xlabel('Prediksi', fontsize=12, fontweight='bold')
plt.ylabel('Aktual', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"\n📊 Detail Confusion Matrix:")
print(f"   - True Negative  (TN): {cm_m3[0][0]}")
print(f"   - False Positive (FP): {cm_m3[0][1]}")
print(f"   - False Negative (FN): {cm_m3[1][0]}")
print(f"   - True Positive  (TP): {cm_m3[1][1]}")

In [ ]:
# Classification Report Model 3
print("="*70)
print("CLASSIFICATION REPORT - MODEL 3")
print("="*70)
print(classification_report(y_test, y_pred_test_m3, 
                           target_names=['Tidak Resign', 'Resign']))

In [ ]:
# Feature Importance Model 3
encoder_m3 = pipeline_model3.named_steps['preprocessor'].named_transformers_['cat']
cat_feature_names_m3 = encoder_m3.get_feature_names_out(cat_features_m3).tolist()
all_feature_names_m3 = cat_feature_names_m3 + num_features_m3

importances_m3 = pipeline_model3.named_steps['classifier'].feature_importances_

feat_imp_m3 = pd.DataFrame({
    'Feature': all_feature_names_m3,
    'Importance': importances_m3
}).sort_values('Importance', ascending=False)

print("="*70)
print("FEATURE IMPORTANCE - MODEL 3 (MINIMAL)")
print("="*70)
print(feat_imp_m3.to_string(index=False))

In [ ]:
# Visualisasi Feature Importance Model 3
fig, ax = plt.subplots(figsize=(10, 6))

top_m3 = feat_imp_m3.sort_values('Importance', ascending=True)
colors = plt.cm.Oranges(np.linspace(0.4, 0.9, len(top_m3)))

bars = ax.barh(top_m3['Feature'], top_m3['Importance']*100, 
               color=colors, edgecolor='black', linewidth=1.5)

for bar, val in zip(bars, top_m3['Importance']*100):
    ax.text(val + 0.3, bar.get_y() + bar.get_height()/2, 
            f'{val:.2f}%', va='center', fontweight='bold', fontsize=11)

ax.set_xlabel('Importance (%)', fontsize=12, fontweight='bold')
ax.set_title('Feature Importance - Model 3 (Minimal Features)\n7 Fitur Paling Penting', 
             fontsize=14, fontweight='bold', pad=15)
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

### 📝 Kesimpulan Model 3 (Minimal Features)

**Hasil:**
- Akurasi Testing: ~84% (hampir sama dengan Model 1 & 2!)
- Hanya menggunakan 7 fitur (pengurangan 77% dari Model 1)
- Waktu training paling cepat

**Insight:**
Model 3 dengan hanya 7 fitur berhasil mencapai akurasi yang **hampir sama** dengan Model 1 yang menggunakan 31 fitur. Ini membuktikan bahwa:
1. Tidak semua fitur diperlukan untuk prediksi yang akurat
2. 7 fitur utama sudah cukup menjelaskan pola attrition
3. Model yang lebih sederhana lebih praktis untuk deployment

---
---
# 📊 BAGIAN 4: PERBANDINGAN SEMUA MODEL

Setelah melatih 3 model dengan jumlah fitur yang berbeda, mari kita bandingkan performanya secara menyeluruh.

In [ ]:
# Tabel Perbandingan Semua Model
print("="*80)
print("PERBANDINGAN PERFORMA SEMUA MODEL")
print("="*80)

comparison_data = {
    'Model': ['Model 1 (Full)', 'Model 2 (Reduced)', 'Model 3 (Minimal)'],
    'Jumlah Fitur': [31, 11, 7],
    'Akurasi Training (%)': [train_acc_m1*100, train_acc_m2*100, train_acc_m3*100],
    'Akurasi Testing (%)': [test_acc_m1*100, test_acc_m2*100, test_acc_m3*100],
    'Overfitting (%)': [(train_acc_m1-test_acc_m1)*100, (train_acc_m2-test_acc_m2)*100, (train_acc_m3-test_acc_m3)*100],
    'Waktu Training (s)': [training_time_m1, training_time_m2, training_time_m3]
}

comparison_df = pd.DataFrame(comparison_data)
print("\n")
print(comparison_df.to_string(index=False))

In [ ]:
# Visualisasi 1: Perbandingan Akurasi
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Chart 1: Akurasi Training vs Testing
models = ['Model 1\n(31 fitur)', 'Model 2\n(11 fitur)', 'Model 3\n(7 fitur)']
train_accs = [train_acc_m1*100, train_acc_m2*100, train_acc_m3*100]
test_accs = [test_acc_m1*100, test_acc_m2*100, test_acc_m3*100]

x = np.arange(len(models))
width = 0.35

bars1 = axes[0].bar(x - width/2, train_accs, width, label='Training', 
                    color='#4ECDC4', edgecolor='black', linewidth=2)
bars2 = axes[0].bar(x + width/2, test_accs, width, label='Testing', 
                    color='#FF6B6B', edgecolor='black', linewidth=2)

for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        axes[0].annotate(f'{height:.2f}%',
                        xy=(bar.get_x() + bar.get_width()/2, height),
                        xytext=(0, 3), textcoords="offset points",
                        ha='center', va='bottom', fontsize=10, fontweight='bold')

axes[0].set_ylabel('Akurasi (%)', fontsize=12, fontweight='bold')
axes[0].set_title('Perbandingan Akurasi Training vs Testing', fontsize=13, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(models)
axes[0].legend(loc='lower right')
axes[0].set_ylim(75, 95)
axes[0].grid(axis='y', alpha=0.3)

# Chart 2: Jumlah Fitur vs Akurasi Testing
colors = ['#FF6B6B', '#FFA500', '#4ECDC4']
features = [31, 11, 7]

scatter = axes[1].scatter(features, test_accs, s=400, c=colors, 
                          edgecolors='black', linewidth=2, zorder=5)

for i, (x, y, m) in enumerate(zip(features, test_accs, ['M1', 'M2', 'M3'])):
    axes[1].annotate(f'{m}\n{y:.2f}%', xy=(x, y), xytext=(0, 15),
                    textcoords='offset points', ha='center', fontsize=10, fontweight='bold')

axes[1].plot(features, test_accs, 'k--', alpha=0.3, linewidth=2)
axes[1].set_xlabel('Jumlah Fitur', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Akurasi Testing (%)', fontsize=12, fontweight='bold')
axes[1].set_title('Efisiensi Fitur: Jumlah vs Akurasi', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(82, 86)

plt.tight_layout()
plt.show()

In [ ]:
# Visualisasi 2: Perbandingan Confusion Matrix
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

cms = [cm_m1, cm_m2, cm_m3]
titles = ['Model 1 (Full - 31 fitur)', 'Model 2 (Reduced - 11 fitur)', 'Model 3 (Minimal - 7 fitur)']
cmaps = ['Blues', 'Greens', 'Oranges']
accs = [test_acc_m1, test_acc_m2, test_acc_m3]

for idx, (ax, cm, title, cmap, acc) in enumerate(zip(axes, cms, titles, cmaps, accs)):
    sns.heatmap(cm, annot=True, fmt='d', cmap=cmap, ax=ax,
                xticklabels=['Tidak', 'Resign'],
                yticklabels=['Tidak', 'Resign'],
                annot_kws={'size': 14, 'weight': 'bold'},
                linewidths=2, linecolor='black')
    ax.set_title(f'{title}\nAkurasi: {acc*100:.2f}%', fontsize=11, fontweight='bold')
    ax.set_xlabel('Prediksi', fontsize=10)
    ax.set_ylabel('Aktual', fontsize=10)

plt.suptitle('Perbandingan Confusion Matrix - Semua Model', fontsize=14, fontweight='bold', y=1.05)
plt.tight_layout()
plt.show()

In [ ]:
# Visualisasi 3: Proses Reduksi Fitur
fig, ax = plt.subplots(figsize=(10, 5))

stages = ['Dataset\nOriginal', 'Setelah\nCleaning', 'Model 2\n(Reduced)', 'Model 3\n(Minimal)']
feature_counts = [35, 31, 11, 7]
colors = ['#FF6B6B', '#FFA500', '#4ECDC4', '#45B7D1']

bars = ax.bar(stages, feature_counts, color=colors, edgecolor='black', linewidth=2)

for bar, count in zip(bars, feature_counts):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{count}', ha='center', va='bottom', fontsize=14, fontweight='bold')

ax.set_ylabel('Jumlah Fitur', fontsize=12, fontweight='bold')
ax.set_title('Proses Reduksi Fitur\nDari 35 Fitur → 7 Fitur (Pengurangan 80%)', 
             fontsize=14, fontweight='bold', pad=15)
ax.set_ylim(0, 40)
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

---
# 📝 KESIMPULAN AKHIR

## Ringkasan Eksperimen

| Model | Jumlah Fitur | Akurasi Testing | Keterangan |
|-------|--------------|-----------------|------------|
| Model 1 | 31 fitur | ~84% | Baseline dengan semua fitur |
| Model 2 | 11 fitur | ~84% | Pengurangan 65% fitur |
| Model 3 | 7 fitur | ~84% | Pengurangan 77% fitur |

## Temuan Utama

1. **Akurasi Stabil**: Ketiga model memiliki akurasi yang **hampir sama** (~84%), meskipun jumlah fitur berbeda jauh.

2. **Fitur Paling Berpengaruh** (dari Model 3):
   - **OverTime** (Lembur) - Faktor terbesar
   - **MonthlyIncome** (Gaji)
   - **Age** (Usia)
   - **TotalWorkingYears** (Pengalaman)
   - **DistanceFromHome** (Jarak)
   - **StockOptionLevel** (Benefit saham)
   - **EnvironmentSatisfaction** (Kepuasan kerja)

3. **Efisiensi**: Model 3 dengan 7 fitur adalah yang **paling efisien** karena:
   - Akurasi sama dengan model kompleks
   - Waktu training paling cepat
   - Lebih mudah diinterpretasi
   - Praktis untuk deployment

## Rekomendasi

Untuk keperluan **prediksi employee attrition**, disarankan menggunakan **Model 3 (Minimal Features)** karena:
- Performa sama bagus dengan model kompleks
- Hanya membutuhkan 7 input data
- Lebih mudah dimaintain dan dijelaskan ke stakeholder

In [ ]:
# Menentukan Model Terbaik
print("="*70)
print("MODEL TERBAIK")
print("="*70)

# Cari model dengan akurasi testing tertinggi
best_acc = max(test_acc_m1, test_acc_m2, test_acc_m3)
if best_acc == test_acc_m3:
    best_model = "Model 3 (Minimal)"
    best_features = 7
elif best_acc == test_acc_m2:
    best_model = "Model 2 (Reduced)"
    best_features = 11
else:
    best_model = "Model 1 (Full)"
    best_features = 31

print(f"\n⭐ Model Terpilih: {best_model}")
print(f"📊 Akurasi Testing: {best_acc*100:.2f}%")
print(f"📊 Jumlah Fitur   : {best_features}")
print(f"\n✅ Model siap untuk deployment!")